In [6]:
from diagrams import Diagram, Cluster
from diagrams.aws.compute import ECS, EC2
from diagrams.onprem.container import Docker

In [7]:
with Diagram("ECS Deployment - 1. Initial State", show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        containers = [Docker("Container 1"), Docker("Container 2")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_initial_state.png'

In [8]:
with Diagram("ECS Deployment - 2. Deploy New Container", show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # Two original containers and one new version container
        containers = [Docker("Container 1 - Old"), Docker("Container 2 - Old"), Docker("Container 3 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_deploy_new_container.png'

In [9]:
with Diagram("ECS Deployment - 3. Decommission Old Container", show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # One old container and one new version container, after decommissioning one old container
        containers = [Docker("Container 1 - Old"), Docker("Container 2 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_decommission_old_container.png'

In [10]:
with Diagram("ECS Deployment - 4. Rollout Continuation", show=False, direction="TB"):
    with Cluster("ECS Cluster"):
        ecs_service = ECS("ECS Service")
        # Two new version containers after decommissioning the last old container
        containers = [Docker("Container 1 - New"), Docker("Container 2 - New")]

    ec2 = EC2("EC2 Capacity")

    ecs_service >> containers
    ec2 >> ecs_service

# This will save the diagram as 'ecs_deployment_rollout_continuation.png'